## 作業

Hint: 人物原始邊框座標 (60, 40), (420, 510)

請根據 Lena 圖做以下處理

- 對明亮度做直方圖均衡處理
- 水平鏡像 + 縮放處理 (0.5 倍)
- 畫出人物矩形邊框

In [2]:
import cv2
import numpy as np

# Using lena.png
img = 'img/lena.png'

# default read as cv2.IMREAD_COLOR(彩色圖)
img_bgr = cv2.imread(img, cv2.IMREAD_COLOR)

In [3]:
# Drawing Rectangle :
img_rect = img_bgr.copy()
# cv2.rectangle(original image, rect-left-up(x,y), rect-right-down(x,y), rect-color, rect-thickenss)
start_point = (60, 40) 
end_point = (420, 510) 
color = (0, 0, 255)    
thickness = 3          
cv2.rectangle(img_rect, start_point, end_point, color, thickness)

while True :
    # display of img_rect, named as 'image with rect'
    cv2.imshow('image with rect', img_rect)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

In [4]:
# Drawing Line :
img_line = img_bgr.copy()
# cv2.rectangle(original image, line-left-up(x,y), line-right-down(x,y), line-color, line-thickenss)
start_point = (60, 40) 
end_point = (420, 510)
color = (0, 0, 255)    
thickness = 3          
cv2.line(img_line, start_point, end_point, color, thickness)

while True :
    # display of img_line, named as 'image with line'
    cv2.imshow('image with line', img_line)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

In [6]:
# Drawing Text :
img_text = img_bgr.copy()
# cv2.rectangle(original image, text, text_coordinate(x,y), font, font_size, text-color, text-thickenss)
text = '(64, 40)' 
text_coordinate = (64, 40)
font = 0
font_size = 1
color = (255, 0, 0)    
thickness = 2          
cv2.putText(img_text, text, text_coordinate, font, font_size, color, thickness)

while True :
    # display of img_text, named as 'image with text'
    cv2.imshow('image with text', img_text)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

## 解法一 :
1. 顏色的操作 (對明亮度做直方圖均衡)
2. 把矩形邊框畫在圖片上之後再做位置上的變換 -> 水平鏡像 + 縮放處理 (0.5 倍)

得到的圖的結果正確，但你不知道新的矩型邊框座標點

In [4]:
# answer 1 :
img_equal = img_bgr.copy()

# transfer color space to HSV image (3D array)
img_equal = cv2.cvtColor(img_equal, cv2.COLOR_BGR2HSV)
# histogram equalization by value(brightness) channel of HSV image (2D array) -> 增強對比
img_equal[:,:,2] = cv2.equalizeHist(img_equal[:,:,2])

# transfer color space return to BGR
img_equal = cv2.cvtColor(img_equal, cv2.COLOR_HSV2BGR)

# cv2.rectangle(original image, line-left-up(x,y), line-right-down(x,y), line-color, line-thickenss)
start_point = (60, 40) 
end_point = (420, 510) 
color = (0, 0, 255)    
thickness = 3          
cv2.rectangle(img_equal, start_point, end_point, color, thickness)

# horizontal flip(水平翻轉) -> 矩形框跟著水平翻轉
# img_bgr[h, w, channel], w reversed which means horizontal flip
img_equal = img_equal[:, ::-1, :]

# resize to 50% of original image -> 矩形框跟著resize
img_equal = cv2.resize(img_equal, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

while True :
    # display of img_equal, named as 'img equal then rect & mirror & zoom'
    cv2.imshow('img equal then rect & mirror & zoom', img_equal)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

## 解法二 :
1. 顏色的操作 (對明亮度做直方圖均衡)
2. 鏡像可以透過四則運算得知
3. 透過建構 transformation matrix 做縮放
4. 把矩型邊框的點與 transformation matrix 相乘就會得到縮放後的位置
5. 畫圖

得到的圖的結果正確，同時也知道新的矩型邊框座標點

In [7]:
# answer 2 :
img_equal = img_bgr.copy()
point1 = [60, 40]    # left-up
point2 = [420, 510]  # right-down

# transfer color space to HSV image (3D array)
img_equal = cv2.cvtColor(img_equal, cv2.COLOR_BGR2HSV)
# histogram equalization by value(brightness) channel of HSV image (2D array) -> 增強對比
img_equal[:,:,2] = cv2.equalizeHist(img_equal[:,:,2])

# transfer color space return to BGR
img_equal = cv2.cvtColor(img_equal, cv2.COLOR_HSV2BGR)

# image : horizontal flip
img_equal = img_equal[:, ::-1, :]

# point1,point2 : horizontal flip
# img_equal.shape[0] means height, img_equal.shape[1] means width (512, 512)
h, w = img_equal.shape[0], img_equal.shape[1]
#print(h,w)
# point1, point2 : after mirror
point1[0] = w - point1[0]  # right-up
point2[0] = w - point2[0]  # left-down
point1[0], point2[0] = point2[0], point1[0]  
#print(point1, point2)

# image : resize
# Transformation Matrix ([[0.5,0,tx], [0,0.5,ty]]), x resize from 1 to 0.5, y resize from 1 to 0.5(don't move)
M = np.float32([[0.5, 0, 0], [0, 0.5, 0]])
fx = 0.5
fy = 0.5
resize_w = int(img_equal.shape[1]*fx)
resize_h = int(img_equal.shape[0]*fy)
img_equal = cv2.warpAffine(img_equal, M, (resize_w, resize_h))

# point1,point2 : resize
# point1,point2 transfer to numpy array, then matrix dot
rect1 = np.float32((point1, point2))
# M.T, then matrix dot
rect2 = np.dot(M.T, rect1)
# point1,point2 transfer to uint8 type
rect3 = rect2.astype('uint8')
point1_new = tuple(rect3[0])
point2_new = tuple(rect3[1])
print('original point1={}, original point2={}'.format(point1, point2))
print('resized point1={}, resized point2={}'.format(point1_new, point2_new))

# drawing rectangle
color = (0, 0, 255)    
thickness = 3     
cv2.rectangle(img_equal, point1_new, point2_new, color, thickness)

while True :
    # display of img_equal, named as 'img equal then rect & mirror & zoom'
    cv2.imshow('img equal then rect & mirror & zoom', img_equal)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

original point1=[92, 40], original point2=[452, 510]
resized point1=(46, 20), resized point2=(226, 255)


##  解法三 (optional)

1. 把矩型邊框用遮罩的方式呈現，使用同樣處理圖片的方式處理遮罩
2. 最後再找遮罩的左上跟右下的點的位置

In [8]:
# answer 3 :
img_equal = img_bgr.copy()
point1 = (60, 40)    # left-up
point2 = (420, 510)  # right-down

img_mask = np.zeros_like(img_equal)  
img_mask[40:510, 60:420, :] = 255

# transfer color space to HSV image (3D array)
img_equal = cv2.cvtColor(img_equal, cv2.COLOR_BGR2HSV)
# histogram equalization by value(brightness) channel of HSV image (2D array) -> 增強對比
img_equal[:,:,2] = cv2.equalizeHist(img_equal[:,:,2])

# transfer color space return to BGR
img_equal = cv2.cvtColor(img_equal, cv2.COLOR_HSV2BGR)

# image : horizontal flip
img_equal = img_equal[:, ::-1, :]

# mask : horizontal flip
img_mask = img_mask[:, ::-1, :]

# image : resize
img_equal = cv2.resize(img_equal, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

# mask : resize
img_mask = cv2.resize(img_mask, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_LINEAR)

# get the index of img_mask height, weight, channel for drawing rextangle
h, w, c = np.where(img_mask)  
point1_new = (min(w), min(h)) # tuple
point2_new = (max(w), max(h)) # tuple
print('original point1={}, original point2={}'.format(point1, point2))
print('resized point1={}, resized point2={}'.format(point1_new, point2_new))

# drawing rectangle
color = (0, 0, 255)    
thickness = 3     
cv2.rectangle(img_equal, point1_new, point2_new, color, thickness)

while True :
    # display of img_equal, named as 'img equal then rect & mirror & zoom'
    cv2.imshow('img equal then rect & mirror & zoom', img_equal)
    
    # waiting until to press any key then close the display window
    if cv2.waitKey(0) : 
        cv2.destroyAllWindows()
        break

original point1=(60, 40), original point2=(420, 510)
resized point1=(46, 20), resized point2=(225, 254)
